In [ ]:
#required packages
library("DBI", lib.loc="~/R/R-3.5.3/library")
library("tidyverse", lib.loc="~/R/R-3.5.3/library")
library("reshape2", lib.loc="~/R/R-3.5.3/library")
library("writexl", lib.loc="~/R/R-3.5.3/library")
library("sqldf", lib.loc="~/R/R-3.5.3/library")


In [6]:
#connect to database and pull data
con <- dbConnect(odbc::odbc(), "WorkForce")
data <-dbGetQuery(con,"SELECT DW.DimRespondent.DW_Id,
       DW.DimProject.ProjectNumber,
	   DW.DimFacility.FacilityName,
	   DW.DimItem.MasterItemNumber AS MasterItem, 
	   CAST(DW.FactItemResponse.PerformanceScore AS numeric (4,2)) AS PerformanceScore

	  FROM  DW.DimRespondent
            INNER JOIN DW.FactItemResponse ON DW.DimRespondent.DW_Id = DW.FactItemResponse.Respondent_DW_Id
			INNER JOIN DW.DimProject ON DW.FactItemResponse.Project_DW_Id = DW.DimProject.DW_Id
            INNER JOIN DW.DimItem ON DW.FactItemResponse.Item_DW_Id = DW.DimItem.DW_Id 
			INNER JOIN DW.DimSurvey ON DW.FactItemResponse.Survey_DW_Id = DW.DimSurvey.DW_Id
			INNER JOIN DW.DimSpecialty ON DW.FactItemResponse.Specialty_DW_Id = DW.DimSpecialty.DW_Id
		    INNER JOIN DW.DimFacility ON DW.FactItemResponse.Facility_DW_Id = DW.DimFacility.DW_Id

	  WHERE ( DW.FactItemResponse.PerformanceScore < 6 )
        AND ( DW.FactItemResponse.PerformanceScore IS NOT NULL )
		AND ( DW.DimSurvey.productName IN ('Business & Industry: Engagement', 'Employee Engagement',
		'Employee Voice: Engagement', 'Nursing Excellence'))
		AND (DW.DimProject.ProjectNumber) IN (
'project1', 'project2', 'project3')

ORDER BY [DW_Id], [MasterItem]")

In [ ]:
#convert to DW_id to DWId
names(data)[1]<-"DWId"

#reduce number of decimals
options(digits = 3)

In [ ]:
#remove duplicate DWId (repeated surveys)
#if duplicates have 2 different responses one will be eliminated
​RESP <-distinct(data, DWId,MasterItem, .keep_all = TRUE)

#Aggregate data - cube_views
Cube_scores <-dcast(RESP,ProjectNumber + FacilityName ~ MasterItem, fun=mean, fill=0)
​Cube_respondents<-dcast(RESP,ProjectNumber + FacilityName ~ MasterItem, value.var="PerformanceScore", fill=0)


In [ ]:
#remove facilities  that don’t meet respondent threshold <=15
resp_fac <-distinct(RESP, DWId,FacilityName,ProjectNumber, .keep_all = FALSE)
resp_fac2 <-summarise(group_by(resp_fac,ProjectNumber,FacilityName), count=n())
resp_fac3<-resp_fac2[resp_fac2[,"count"]>=15,]


In [ ]:
#merge cube_respondents with resp_fac3 to obtain new dataframe
New_RESP<-sqldf("SELECT * FROM resp_fac3 LEFT JOIN Cube_respondents USING(ProjectNumber,FacilityName)")
New_RESP = subset(New_RESP, select = -c(count))

In [ ]:
#Using new dataframe (New_RESP) determine which Master Items do not meet threshold <=10 per facility
#Obtain new dataframe with accepted Master Items
resp_MI<-melt(New_RESP, id.vars= c('ProjectNumber','FacilityName'),variable.name='MasterItem',value.name='Resp_count')
resp_MI$MasterItem <- as.numeric(as.character(resp_MI$MasterItem))
resp_MI2<-summarize(group_by(resp_MI[resp_MI[,"Resp_count"] > 0,], MasterItem), count=n())
resp_MI3<-resp_MI2[resp_MI2[,"count"]>=10,]


In [ ]:
#Using new dataframe (Cube_scores) determine which Master Items do not meet threshold <=5 per project
#Obtain new dataframe with accepted Master Items
Cube_respondents_proj<-dcast(RESP,ProjectNumber ~ MasterItem, value.var="PerformanceScore", fill=0)
resp_proj<-melt(Cube_respondents_proj, id.vars= c('ProjectNumber'),variable.name='MasterItem',value.name='Resp_count')
resp_proj$MasterItem <- as.numeric(as.character(resp_proj$MasterItem))
resp_proj2<-summarize(group_by(resp_proj[resp_proj[,"Resp_count"] > 0,], MasterItem), count=n())
resp_proj3<-resp_proj2[resp_proj2[,"count"]>=5,]


In [ ]:
#facilties to remove
FacititesToRemove<-resp_fac2[resp_fac2[,"count"]<15,]
FacititesToRemove<-subset(FacititesToRemove, select = -c(count))

#MI to remove per facilty
MiToRemoveFac<-resp_MI2[resp_MI2[,"count"]<10,]
MiToRemoveFac<-subset(MiToRemoveFac,select= -c(count))

#MI to remove per project
MiToRemoveProj<-resp_proj2[resp_proj2[,"count"]<5,]
MiToRemoveProj<-subset(MiToRemoveProj,select= -c(count))

In [ ]:
#Merge MIToRemoveFac and MIToRemoveProj into MiToRemove
#Remove Duplicates
#sort
MIToRemove <-union(MiToRemoveProj,MiToRemoveFac)
MIToRemove <-MIToRemove[order(MIToRemove$MasterItem),]

In [ ]:
#Create final dataset
#Remove facilities + MI that did not meet threshold
final_data <- RESP[apply(RESP, 1, function(r, A) { sum(A$ProjectNumber==r["ProjectNumber"] & A$FacilityName==r["FacilityName"])}, FacititesToRemove) == 0,]
final_data <- final_data[!apply(final_data, 1, function(r, A) {as.numeric(r["MasterItem"]) %in% A}, MIToRemove$MasterItem),]

In [ ]:
#Average Performance Score per MI Output
Final_Cube_Scores <-summarise(group_by(final_data, MasterItem), PerfScore=mean(PerformanceScore))
write_xlsx(x = Final_Cube_Scores, path = "C:\\Users\\sotom\\Desktop\\RTempOutput\\Final_Cube_Scores.csv", col_names = TRUE)

#New cubes
New_Cube_scores <-dcast(final_data,ProjectNumber + FacilityName ~ MasterItem, fun=mean, fill=0)
New_Cube_respondents<-dcast(final_data,ProjectNumber + FacilityName ~ MasterItem, value.var="PerformanceScore", fill=0)
write_xlsx(x = New_Cube_scores, path = "C:\\Users\\sotom\\Desktop\\RTempOutput\\New_Cube_scores.csv", col_names = TRUE)
write_xlsx(x = New_Cube_respondents, path = "C:\\Users\\sotom\\Desktop\\RTempOutput\\New_Cube_respondents.csv", col_names = TRUE)

#project count per MI before filters
write_xlsx(x = resp_proj3, path = "C:\\Users\\sotom\\Desktop\\RTempOutput\\ProjectCountperMI.csv", col_names = TRUE)

#end facility count per MI
write_xlsx(x = resp_MI3, path = "C:\\Users\\sotom\\Desktop\\RTempOutput\\FacilityCountperMI.csv", col_names = TRUE)